In [1]:
# создание хэш-таблицы
from Bio import SeqIO

# file = input('Введите название fasta-файла')
# k = int(input())

file = 'pseudomonas_aeruginosa.fasta'
k = 11

seq = list(SeqIO.parse(file, 'fasta'))[0].seq
len_seq = len(seq)
k_dict = dict()

# делаем словарь - хэш-таблицу
for i in range(len(seq) - k + 1):
    k_mer = str(seq[i:i+k])
    # добавляем к координатам ведущие 0, это нам еще понадобится
    i = (len(str(len_seq)) - len(str(i))) * '0' + str(i)
    if k_mer not in k_dict:
        k_dict[k_mer] = [i]
    else:
        k_dict[k_mer].append(i)

# записываем в текстовый файл
f = open("hash.txt", "w")
for elem in k_dict.items():
    f.write(str(elem[0]) + ': ' + str(elem[1]).strip('[]') + '\n')
f.close()

In [ ]:
# картирование ридов на геном
from Bio.Align import PairwiseAligner

aligner = PairwiseAligner()
aligner.mode = "global"
aligner.match_score = 5
aligner.mismatch_score = -4
aligner.open_gap_score = -10
aligner.extend_gap_score = -10


# fastq = input('Введите название fastq-файла')
# https://digitalinsights.qiagen.com/downloads/example-data/ - ссылка для скачивания файла-примера с ридами
fastq = 'SRR396636.sra_1.fastq'
seq = list(SeqIO.parse(file, 'fasta'))[0].seq

# будущий файл с картированием
mapping = open('mapping.txt', 'w')

# распаковываем хэш-таблицу из файла
hash_table = dict()
with open('hash.txt', 'r') as file:
    for line in file:
        line = line.split(': ')
        hash_table[line[0]] = line[1].strip("\n").split(', ')

# для каждого рида создаем словарь с его k-мерами в ключах и координатами этих k-меров
# в геноме в значениях (координаты берем из хэш-таблицы)

for record in SeqIO.parse(fastq, 'fastq'):
    r_dict = dict()
    all_locations = []
    all_locations_prefix = []
    read = record.seq
    for i in range(len(read) - k + 1):
        k_mer = read[i:i+k]
        if k_mer in hash_table.keys():
            r_dict[k_mer] = hash_table[str(k_mer)]
    # теперь нужно понять, как картировать рид на геном
    # нам известна длина ридов, поэтому если k-меры принадлежат одной локации,
    # то их координаты без трех последних символов будут совпадать
    for elem in r_dict.values():
        for e in elem:
            all_locations.append(e.strip("'"))
            all_locations_prefix.append(e[0:len(e) - 4].strip("'"))  # 4, т.к. там еще есть кавычка

    # срдеи всех локаций находим самую часто встречающуюся
    all_locations_set = set(all_locations_prefix)
    most_common = None
    qty_most_common = 0
    for loc in all_locations_set:
        qty = all_locations_prefix.count(loc)
        if qty > qty_most_common:
            qty_most_common = qty
            most_common = loc

    # будем считать картирование удачным, если его качество (количество идеально попавших k-меров) выше 80%
    q = all_locations_prefix.count(most_common) / (len(read) - k + 1)
    if q > 0.8:
        coords = list(map(int, [elem for elem in all_locations if elem[0:len(elem) - 3] == most_common]))
        mapping.write('\n')
        mapping.write('read\n')
        mapping.write(str(aligner.align(read, seq[min(coords): min(coords) + len(read)])[0]))
        mapping.write('genome' + '\n')
        mapping.write('Coordinates: ' + str(min(coords)) + ':' + str(min(coords) + len(read)) + '\n')
